# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: **SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Joaquín De Groote
- Nombre de alumno 2: Vicente Pinochet R.

### **Link de repositorio de GitHub:** [Insertar Enlace](https://github.com/Qajirr/MDS7202-Labs)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [ ]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [ ]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

`escriba su respuesta acá`

**Blackjack - Descripción del Ambiente**

El objetivo en **Blackjack** es vencer al dealer obteniendo cartas que sumen lo más cerca posible a 21, sin pasarse. El juego comienza con el dealer mostrando una carta y otra oculta, mientras el jugador recibe dos cartas visibles. Las cartas se sacan de un mazo infinito.

**Valores de las cartas**:
- Cartas de figuras (J, Q, K): **10 puntos**.
- Ases: **1 o 11 puntos** (usable ace).
- Cartas numéricas (2-9): Valor numérico.

**Acciones**:
- **0 (Stick)**: Detenerse.
- **1 (Hit)**: Pedir una carta.

**Espacio de observación**:
- Tupla \((suma\_jugador, carta\_dealer, as\_usable)\):
  - **suma\_jugador**: 4-21.
  - **carta\_dealer**: 1-10.
  - **as\_usable**: 0 o 1.

**Recompensas**:
- **+1**: Victoria.
- **0**: Empate.
- **-1**: Derrota.
- **+1.5**: Blackjack natural (opcional).

**Fin del episodio**:
- Cuando el jugador se pasa de 21 o elige "Stick".


#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [ ]:
# Simulación de 5000 episodios con acciones aleatorias
num_episodes = 5000
rewards = []

for _ in range(num_episodes):
    observation, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Tomar una acción aleatoria (0: Stick, 1: Hit)
        action = env.action_space.sample()
        observation, reward, done, _, _ = env.step(action)
        total_reward += reward
    
    rewards.append(total_reward)

# Calcular estadísticas
mean_reward = np.mean(rewards)
std_reward = np.std(rewards)

El promedio negativo de recompensas **-0.40** sugiere que la política de tomar acciones aleatorias resulta en más derrotas que victorias. Esto implica que el jugador pierde más de lo que gana en promedio. La desviación estándar de **0.89** muestra una considerable variabilidad en los resultados, lo que indica que las recompensas fluctúan significativamente de un episodio a otro.

**Conclusión**: El performance de esta política aleatoria es pobre, ya que genera pérdidas más frecuentemente. Para mejorar, se necesitaría una estrategia que considere la probabilidad de ganar y optimice las acciones en consecuencia.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [ ]:
from stable_baselines3 import DQN
# Crear el modelo DQN
model = DQN("MlpPolicy", env, verbose=1)

# Entrenar el modelo durante 10000 pasos
model.learn(total_timesteps=10000)

# Guardar el modelo entrenado
model.save("dqn_blackjack_model")

# Evaluar el rendimiento del modelo
episodes = 1000
rewards = []

for _ in range(episodes):
    observation, _ = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action, _ = model.predict(observation, deterministic=True)
        observation, reward, done, _, _ = env.step(action)
        total_reward += reward
        
    rewards.append(total_reward)

mean_reward = np.mean(rewards)
print(f"Recompensa promedio después del entrenamiento: {mean_reward:.2f}")

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
model = DQN.load("dqn_blackjack_model")

# Evaluar el rendimiento del modelo entrenado
episodes = 1000
rewards_trained_model = []

for _ in range(episodes):
    observation, _ = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        # Predecir la acción con el modelo entrenado
        action, _ = model.predict(observation, deterministic=True)
        observation, reward, done, _, _ = env.step(action)
        total_reward += reward
        
    rewards_trained_model.append(total_reward)

# Calcular las estadísticas para el modelo entrenado
mean_reward_trained = np.mean(rewards_trained_model)
std_reward_trained = np.std(rewards_trained_model)

El promedio negativo de recompensas **-0.06** sugiere que el modelo entrenado aún tiene un rendimiento ligeramente negativo, con un equilibrio entre victorias y derrotas. Esto implica que, aunque el agente ha aprendido a jugar, no genera ganancias consistentes en promedio. La desviación estándar de  **0.96** muestra una considerable variabilidad en los resultados, lo que indica que las recompensas fluctúan significativamente de un episodio a otro.

**Conclusión**: El performance del modelo entrenado es mejor que el de la política aleatoria, pero aún no es ideal. Para mejorar, se podría optimizar el modelo, ajustar parámetros o explorar diferentes estrategias de aprendizaje para incrementar la tasa de victorias.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [ ]:
initial_state = env.reset()

# Verificar el tipo del estado
print(f"Tipo de estado: {type(initial_state)}")

In [ ]:
initial_state

In [ ]:
# Función para obtener la acción del agente dada una observación
def get_agent_action(state):
    action, _ = model.predict(state, deterministic=True)
    return action

# Escenario 1: Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene un as
# Estado: (6, 7, 0)
state_1 = np.array([6, 7, 0])

# Escenario 2: Suma de cartas del agente es 19, dealer muestra un 3, agente tiene un as
# Estado: (19, 3, 1)
state_2 = np.array([19, 3, 1])

# Obtener las acciones para ambos estados
action_1 = get_agent_action(state_1)
action_2 = get_agent_action(state_2)

# Mostrar las acciones
print(f"Acción para el escenario 1 (estado {state_1}): {'Hit' if action_1 == 1 else 'Stick'}")
print(f"Acción para el escenario 2 (estado {state_2}): {'Hit' if action_2 == 1 else 'Stick'}")

Escenario 1: La suma de cartas del agente es 6, el dealer muestra un 7 y el agente no tiene un as.

- Regla general: Cuando el agente tiene una suma baja, es probable que deba "hit" (pedir más cartas) hasta llegar a una suma más cercana a 21.
- Esperado: El modelo debería elegir hit.

Escenario 2: La suma de cartas del agente es 19, el dealer muestra un 3, y el agente tiene un as.

- Regla general: Con una suma alta (como 19), el agente debería plantarse (stick). Tener un as como "usable ace" podría hacer que el agente sea un poco más flexible, pero aún así, la estrategia general es plantarse.
- Esperado: El modelo debería elegir stick.

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [ ]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

`escriba su respuesta acá`

### **Descripción de MDP en LunarLander-v2**

El entorno **LunarLander-v2** simula un problema de optimización de la trayectoria de un cohete, donde el objetivo es aterrizar el módulo en la luna. La formulación en MDP es la siguiente:

#### **Estado (S)**
El estado es un vector de 8 dimensiones que incluye:
- **x, y**: Coordenadas del módulo.
- **vx, vy**: Velocidades en los ejes horizontal y vertical.
- **ángulo**: Orientación del módulo.
- **velocidad angular**: Rotación del módulo.
- **contacto con piernas**: Booleanos que indican si las piernas están tocando el suelo.

#### **Acciones (A)**
Existen 4 acciones posibles:
- **0**: No hacer nada.
- **1**: Encender el motor izquierdo (orientación).
- **2**: Encender el motor principal (vertical).
- **3**: Encender el motor derecho (orientación).

#### **Recompensas (R)**
La recompensa depende de la posición, velocidad, y orientación del módulo:
- Posicionarse cerca de la plataforma y reducir la velocidad es premiado.
- Se penaliza la inclinación del módulo.
- **+10 puntos por pierna** si toca el suelo.
- Penalizaciones por el uso de motores: **-0.03** por cada cuadro con motores laterales encendidos, **-0.3** por el motor principal.
- **+100 puntos** por aterrizaje seguro, **-100 puntos** por estrellarse.

#### **Comparación con Blackjack**
A diferencia de **Blackjack**, las acciones en **LunarLander** son físicas, controlando motores para afectar la posición, velocidad y orientación en un entorno 2D. En **Blackjack**, las decisiones son más simples y se basan en probabilidades de cartas. Además, en LunarLander las acciones se manejan de forma discreta pero más compleja.


#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [ ]:
# Función para ejecutar una política aleatoria y obtener las recompensas
def simulate_random_policy(env, episodes=10):
    rewards = []
    for _ in range(episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = env.action_space.sample()  # Acción aleatoria
            obs, reward, done, _, _ = env.step(action)
            total_reward += reward
        rewards.append(total_reward)
    return rewards

# Simulación y cálculo de estadísticas
rewards = simulate_random_policy(env, episodes=10)
mean_reward = np.mean(rewards)
std_reward = np.std(rewards)

El promedio negativo de recompensas **-237.81** sugiere que la política aleatoria tiene un rendimiento deficiente, con una tendencia a las pérdidas. Esto implica que, al no optimizar las acciones, el agente pierde más de lo que gana en promedio. La desviación estándar de **108.99** muestra una considerable variabilidad en los resultados, lo que indica que las recompensas fluctúan significativamente de un episodio a otro.

**Conclusión**: El performance de esta política aleatoria es pobre y no permite ganar consistentemente. Se necesita una estrategia más optimizada para mejorar el rendimiento en este entorno.

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [ ]:
from stable_baselines3 import PPO
# Definir y entrenar el modelo con PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Guardar el modelo entrenado
model.save("ppo_lunarlander_10000_steps")

# Evaluar el modelo entrenado
obs, _ = env.reset()
done = False
total_rewards = 0

# Realizar una evaluación para ver cómo se desempeña el modelo
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, _, _ = env.step(action)
    total_rewards += reward

print(f"Recompensa total al final del episodio: {total_rewards}")

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
# Cargar el modelo entrenado
model = PPO.load("ppo_lunarlander_10000_steps",env)

# Número de episodios para evaluar
num_episodes = 10
rewards = []

# Evaluación del modelo en varios episodios
for _ in range(num_episodes):
    obs, info = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward
    
    rewards.append(total_reward)

# Calcular el promedio y desviación estándar de las recompensas
mean_reward_trained = np.mean(rewards)
std_reward_trained = np.std(rewards)

El promedio de recompensas del modelo entrenado es **-189.59**, lo que sugiere que el modelo tiene un rendimiento moderado, aunque no es perfecto. Esto indica que el agente ha aprendido a optimizar sus acciones, pero aún podría mejorar en algunos aspectos. La desviación estándar de **165.69** muestra que las recompensas fluctúan entre episodios, lo que sugiere cierta variabilidad en el desempeño del agente.

**Comparación con la política baseline**: Comparando este rendimiento con la política baseline (acción aleatoria), podemos concluir que el modelo entrenado tiene un rendimiento mucho mejor, ya que la política aleatoria generaba un promedio de **-206.93** recompensas, mientras que el modelo entrenado está significativamente por encima de ese valor.

**Conclusión**: El rendimiento del agente entrenado es superior al de la política baseline, pero aún existen oportunidades para mejorar. El agente ha aprendido a jugar, pero su desempeño podría optimizarse con más entrenamiento o ajustes en los hiperparámetros del modelo.

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [ ]:
# Definir parámetros para la optimización
total_timesteps = 80000
learning_rate = 0.0003  # Ajuste del learning rate
batch_size = 64  # Aumento del tamaño del batch

# Definir y entrenar el modelo con PPO
model = PPO("MlpPolicy", env, learning_rate=learning_rate, batch_size=batch_size, verbose=1)
model.learn(total_timesteps=total_timesteps)

# Guardar el modelo entrenado
model.save("ppo_lunarlander_opt")

# Evaluar el modelo entrenado
obs, _ = env.reset()
done = False
total_rewards = 0

# Realizar una evaluación
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, truncated, _ = env.step(action)
    total_rewards += reward
    
    # Si el episodio termina o es truncado, detener
    if done or truncated:
        break

print(f"Recompensa total al final del episodio: {total_rewards}")


In [ ]:
# Cargar el modelo entrenado
model = PPO.load("ppo_lunarlander_opt",env)

# Número de episodios para evaluar
num_episodes = 5
rewards = []

for _ in range(num_episodes):
    obs, info = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward
        
        # Si el episodio termina o es truncado, detener
        if done or truncated:
            break

    rewards.append(total_reward)

# Calcular el promedio y desviación estándar de las recompensas
mean_reward_trained = np.mean(rewards)
std_reward_trained = np.std(rewards)

print(f"Promedio de recompensas: {mean_reward_trained:.2f}")
print(f"Desviación estándar de recompensas: {std_reward_trained:.2f}")

In [ ]:
# Crear un GIF del comportamiento del agente optimizado
export_gif(model)

El promedio de recompensas del modelo optimizado es **67.74**, lo que sugiere que el agente ha mejorado significativamente su desempeño, alcanzando un rendimiento superior a 50 recompensas en promedio. Esto indica que el agente ha aprendido a optimizar sus acciones de manera efectiva. La desviación estándar de **49.35** muestra que las recompensas son más consistentes que antes, lo que refleja una mejora en la estabilidad del agente.

**Comparación con el modelo anterior**: En comparación con el modelo entrenado previamente, el modelo optimizado ha mostrado una mejora clara en el rendimiento, ya que las recompensas promedio eran inferiores a 50 en el modelo anterior.

**Conclusión**: El rendimiento del modelo optimizado es mucho mejor. Los ajustes en los parámetros han permitido al agente aprender de manera más efectiva y mejorar su desempeño en el entorno de LunarLander.

![Agente optimizado](agent_performance.gif)

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [2]:
%pip install --upgrade --quiet PyPDF2
# Instalar bibliotecas necesarias si no están instaladas
%pip install --quiet langchain
%pip install --quiet PyPDF2 sentence-transformers faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import PyPDF2

doc_paths = [] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

AssertionError: Deben adjuntar un mínimo de 2 documentos

In [14]:
# Lista de rutas a los documentos PDF
doc_paths = [
    r"C:\Users\joaqu\Downloads\c_1925.pdf",
    r"C:\Users\joaqu\Downloads\constitucion_chile.pdf"
]

# Validar número de documentos y páginas
import PyPDF2

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

# Calcular el total de páginas
total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"
print(f"Total de páginas: {total_paginas}")

Total de páginas: 153


#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [16]:
# Instalar bibliotecas necesarias
%pip install --quiet PyPDF2 sentence-transformers faiss-cpu

# Importar librerías necesarias
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
import pickle

# Función para dividir texto en fragmentos manualmente
def dividir_texto(text, chunk_size, chunk_overlap):
    fragments = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        fragments.append(text[start:end])
        start += chunk_size - chunk_overlap
    return fragments

# Función para cargar y vectorizar documentos
def cargar_y_vectorizar_documentos(doc_paths, chunk_size=1000, chunk_overlap=100):
    # Diccionario para almacenar los textos de cada documento
    doc_texts = {}

    # Extraer texto de los documentos PDF
    for doc in doc_paths:
        with open(doc, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            text = "".join(page.extract_text() for page in reader.pages)
            doc_texts[doc] = text  # Guardar el texto extraído en el diccionario

    # Dividir textos en fragmentos manejables
    texts = []  # Lista para almacenar los fragmentos
    metadatas = []  # Lista para los metadatos

    for doc_name, text in doc_texts.items():
        split_text = dividir_texto(text, chunk_size, chunk_overlap)  # Fragmentar el texto
        texts.extend(split_text)  # Agregar los fragmentos a la lista
        metadatas.extend([{"source": doc_name}] * len(split_text))  # Asociar metadatos a cada fragmento
        print(f"Documento: {doc_name} - Fragmentos generados: {len(split_text)}")

    # Usar SentenceTransformers para generar embeddings
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(texts, show_progress_bar=True)
    print(f"Dimensiones de los embeddings: {embeddings.shape}")

    # Crear índice FAISS basado en similitud L2
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)  # Añadir los embeddings al índice

    # Guardar índice FAISS y metadatos
    with open("vector_store.pkl", "wb") as f:
        pickle.dump({"index": index, "texts": texts, "metadatas": metadatas}, f)

    return index, len(texts)

# Rutas de documentos (asegúrate de haber subido estos archivos)
doc_paths = ["c_1925.pdf", "constitucion_chile.pdf"]  # Cambia según tus archivos

# Ejecutar el proceso de vectorización
index, total_chunks = cargar_y_vectorizar_documentos(doc_paths)

print(f"Vectorización completa. Total de fragmentos generados: {total_chunks}")
print("Los vectores y metadatos se han almacenado en 'vector_store.pkl'.")


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Documento: c_1925.pdf - Fragmentos generados: 76
Documento: constitucion_chile.pdf - Fragmentos generados: 303


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Dimensiones de los embeddings: (379, 384)
Vectorización completa. Total de fragmentos generados: 379
Los vectores y metadatos se han almacenado en 'vector_store.pkl'.


#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

`escriba su respuesta acá`

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [ ]:
%pip install --upgrade --quiet gradio

Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )